In [1]:
import time
import numpy as np
import pandas as pd
import ringity as rng
import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from itertools import product
from ipywidgets import widgets
from collections import Counter
from scipy.spatial.distance import squareform
from ringity.classes.network_model import NetworkBuilder

## Testing expected conditional probability and similairty

In [2]:
def expected_conditional_similarity(theta, rate, a):
    if np.isclose(rate, 0, atol=1e-7):
        return a
    
    if theta < np.pi:
        if theta <= 2*np.pi*a:
            norm = 1 / (1 - np.exp(-2*np.pi*rate)) / (2*a*np.pi*rate)
            
            theta_term = -theta*rate * (1 - np.exp(-2*np.pi*rate))
            
            term1  = -2*np.exp(-rate*theta) + 1
            term21 =  np.exp(-rate*theta)*(np.exp(-2*a*np.pi*rate) + np.exp((2*a*np.pi - 2*np.pi)*rate))
            term22 = -np.exp(-2*np.pi*rate)
            
            return norm*(term1 + term21 + term22 + theta_term) + 1
        else:
            theta_term = np.exp(-rate*theta) / (1 - np.exp(-2*np.pi*rate))
            alpha_term = (np.cosh(-rate*2*np.pi*a) - 1) / (a*np.pi*rate)
            return theta_term * alpha_term
        
    else:
        norm = 2*rate*np.exp(-rate*theta) / (1 - np.exp(-2*np.pi*rate))
        
        def undef_int(t):
            return (np.exp(np.pi*rate)*(np.cosh((np.pi - t)*rate) + (-2*a*np.pi + t)*rate*np.sinh((np.pi - t)*rate)))/(2*a*np.pi*rate**2)
        
        if 2*np.pi - theta <= 2*np.pi*a:
            term1 = (-1 + np.cosh((2*np.pi - theta)*rate) + (2*(-1 + a)*np.pi + theta)*rate*np.sinh((2*np.pi - theta)*rate))/(2*a*np.pi*rate**2)
            term2 = undef_int(2*np.pi*a) - undef_int(2*np.pi - theta)
            return norm*term1 + norm*term2
        else:
            return norm*(np.sinh(a*np.pi*rate)**2)/(a*np.pi*rate**2)

In [3]:
def time_estimation(i, iter_len):
    TIME2 = time.time()
    t_cur = TIME2 - TIME1
    prog = (i+1)/iter_len
    t_rem = t_cur * (1/prog - 1)
    print(f"time passed: {t_cur:.2f}sec - progress: {100*prog:.2f}% - time remaining: {t_rem:.2f}sec",
          end = '\r')

In [4]:
N = 2**10

df_list = []

n_a_samples = 2**4
n_b_samples = 2**4
n_k_samples = 2**4

a_arr = np.linspace(0.01, 0.5, n_a_samples)
b_arr = np.linspace(0.50, 1.0, n_b_samples)
k_arr = np.linspace(0, 1, n_k_samples)

TIME1 = time.time()
count = 0
for alpha, beta, K in product(a_arr, b_arr, k_arr):
    time_estimation(count, n_a_samples*n_b_samples*n_k_samples)
    count += 1
    
    rate = rng.classes.network_model.get_rate_parameter(rate = None, beta = beta)
    scale = 1/rate if rate > 0 else np.inf

    random_positions = []

    position_generator = NetworkBuilder()
    position_generator.set_distribution('exponential', scale = scale)

    for _ in range(10):
        position_generator.instantiate_positions(N)
        random_positions.extend(position_generator.positions)

    deg_dist0_exp = [p for theta in random_positions
                       for p in np.random.binomial(N-1, K*expected_conditional_similarity(theta=theta, rate=rate, a=alpha), size=100)]
    degs_exp, freqs_exp = zip(*Counter(deg_dist0_exp).items())
    freqs_exp = np.array(freqs_exp) / sum(freqs_exp)

    df_list.append(pd.DataFrame({'degree' : degs_exp,
                                 'frequency' : freqs_exp,
                                 'N' : N,
                                 'beta' : beta,
                                 'alpha' : alpha,
                                 'K' : K}
                               )
                  )

In [5]:
plot_df = pd.concat(df_list).sort_values('degree')

# Widget

In [6]:
eps = 0.01

def get_curr_plot_df(plot_df):
    curr_df = (plot_df[
                    plot_df.
                        beta.
                        between(beta_slider.value - eps,
                                beta_slider.value + eps) 
                        &
                    plot_df.
                        alpha.
                        between(alpha_slider.value - eps,
                                alpha_slider.value + eps)
                        &
                    plot_df.
                        K.
                        between(K_slider.value - eps,
                                K_slider.value + eps)].
                 groupby('degree').
                 mean().
                 reset_index())
    return curr_df

def response(change):
    curr_df = get_curr_plot_df(plot_df)
    with w_fig.batch_update():
        w_fig.data[0].x = curr_df.degree
        w_fig.data[0].y = curr_df.frequency

In [7]:
betas_step = 2**4 - 1
alphas_step = 2**4 - 1
K_step = 2**4 - 1

beta_slider = widgets.FloatSlider(
                            value = max(b_arr),
                            max = max(b_arr),
                            min = min(b_arr),
                            step = (max(b_arr) - min(b_arr)) / betas_step,
                            description = 'beta'
                        )

alpha_slider = widgets.FloatSlider(
                            value = max(a_arr),
                            max = max(a_arr),
                            min = min(a_arr),
                            step = (max(a_arr) - min(a_arr)) / alphas_step,
                            description = 'a'
                        )

K_slider = widgets.FloatSlider(
                            value = max(k_arr),
                            max = max(k_arr),
                            min = min(k_arr),
                            step = (max(k_arr) - min(k_arr)) / K_step,
                            description = 'K'
                        )

container = widgets.VBox([beta_slider, alpha_slider, K_slider])

In [12]:
curr_plot_df = get_curr_plot_df(plot_df)
trace = go.Scatter(x = curr_plot_df.degree, y =  curr_plot_df.frequency, name = 'degree distribution')
w_fig = go.FigureWidget(data = trace,
                          layout = go.Layout(title = {'text' : f'Degree distribution under different parameters for N={N}'})
                   )

w_fig.layout.xaxis.title = 'degree'
w_fig.layout.yaxis.title = 'frequency'
w_fig.layout.xaxis.range = (-1, 900)
w_fig.layout.yaxis.range = (-0.005, 0.05)

In [13]:
beta_slider.observe(response, 
                    names = "value")
alpha_slider.observe(response, 
                    names = "value")
K_slider.observe(response, 
                    names = "value")

In [14]:
widgets.VBox([beta_slider, alpha_slider, K_slider, w_fig])

In [15]:
plot_df.to_csv("/Users/dottolab/Desktop/deg_dist2.csv")